First exploration of a new strategy: Inside Bar Momentum.  Simply:  if high-low of bar 2 within high-low of bar 1: signal...

In [43]:
import pandas as pd
import plotly.graph_objects as go
import utils

We know we are going to look for data on USD_JPY_H4, but we haven't yet tested collect_his_data with this pairing, so we need to run that first.  Might as well refresh while we're in the area...

But, this takes a long time to run - so comment out after doing it once.

In [44]:
# import os
# %run ./collect_his_data.py


In [45]:
def plot_candles_mode(df_plot, mode=None):

    if mode == None:
        mode = "markers"

    plot_colours_buy = ['#043ef9', '#eb5334', '#34eb37']
    plot_colours_sell = ['white', 'red', 'yellow']

    fig = go.Figure()
    
    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low = df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor="#24A06B",
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color="#2EC886",
        decreasing_line_color="#FF3A4C"
        ))

    # Loop through buys, and plot
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_buys.time,
            y=df_buys[plot_cols[i]],
            mode=mode,
            name=(f"Buy {plot_cols[i]}"),
            marker=dict(color=plot_colours_buy[i], size=12),
            
        ))

    # Loop through sells and plot
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_sells.time,
            y=df_sells[plot_cols[i]],
            mode=mode,
            name=(f"Sell {plot_cols[i]}"),
            marker=dict(color=plot_colours_sell[i], size=12)
        ))


    fig.update_layout(width=1000, height=400, paper_bgcolor = "#1e1e1e", plot_bgcolor = "#1e1e1e",
                    margin=dict(l=10, b=10, t=30, r=10), 
                    font=dict(size=10, color="#e1e1e1"),
                    title=f"Buy/Sell {mode} Chart")
    fig.update_xaxes(gridcolor="#1f292f",
                    showgrid=True,
                    fixedrange=True,
                    rangeslider=dict(visible=False),
                    rangebreaks=[
                        dict(bounds=["sat", "mon"])
                        ]
                    )
    fig.update_yaxes(gridcolor="#1f292f",
                    showgrid=True)

    fig.show()

In [46]:
pair = "GBP_JPY"
granularity = "M5"
df_raw = pd.read_csv(utils.get_hist_data_filename(pair, granularity))


In [47]:
df_raw.describe()

,Unnamed: 0,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,mid_l,mid_c,ask_o,ask_h,ask_l,ask_c
count,224097.000000,224097.000000,224097.000000,224097.000000,224097.000000,224097.000000,224097.000000,224097.000000,224097.000000,224097.000000,224097.000000,224097.000000,224097.000000,224097.000000
mean,709.358354,474.704972,149.962573,150.000914,149.923802,149.962783,149.979455,150.017528,149.941182,149.979550,149.996339,150.034931,149.957796,149.996316
std,410.497432,599.004297,10.957041,10.960459,10.953283,10.957071,10.957420,10.960785,10.953683,10.957411,10.957813,10.961207,10.954028,10.957764
min,0.000000,1.000000,124.492000,124.782000,123.969000,124.488000,124.567000,124.857000,124.044000,124.563000,124.642000,124.932000,124.119000,124.638000
25%,354.000000,107.000000,139.614000,139.642000,139.586000,139.614000,139.631000,139.658000,139.604000,139.632000,139.646000,139.674000,139.619000,139.646000
50%,709.000000,238.000000,151.660000,151.687000,151.630000,151.660000,151.674000,151.702000,151.644000,151.673000,151.686000,151.715000,151.657000,151.686000
75%,1063.000000,622.000000,158.688000,158.751000,158.617000,158.689000,158.706000,158.768000,158.636000,158.704000,158.723000,158.787000,158.654000,158.721000
max,1439.000000,27801.000000,172.069000,172.118000,171.996000,172.074000,172.089000,172.136000,172.016000,172.092000,172.109000,172.153000,172.035000,172.110000


In [48]:
non_nums = ['time', 'volume']
num_cols = [x for x in df_raw.columns if x not in non_nums]
df_raw[num_cols] = df_raw[num_cols].apply(pd.to_numeric)



In [49]:
df_raw.columns

Index(['Unnamed: 0', 'time', 'volume', 'bid_o', 'bid_h', 'bid_l', 'bid_c',
       'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [50]:
df = df_raw[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c']].copy()

In [51]:
def direction(row):
    """ Return price direction based on close vs open price i.e. if close higher, upwards (1)..."""
    if row.mid_c > row.mid_o:
        return 1
    return -1

In [52]:
def direction(row):
    """ Return price direction based on close vs open price i.e. if close higher, upwards (1)..."""
    if row.mid_c > row.mid_o:
        return 1
    return -1

In [53]:
SLOSS = 0.4
TPROFIT = 0.8
ENTRY_PRC = 0.1

def direction(row):
    """ Return price direction based on close vs open price i.e. if close higher, upwards (1)..."""
    if row.mid_c > row.mid_o:
        return 1
    return -1

def get_signal(row):
    """  Identify encapsulation in previous candle and return direction.  Pass back zero if no encapsulation.  
    1 = Buy, -1=Sell:  Buy if encapsulated in a previously upward candle...      """
    if row.mid_h_prev > row.mid_h and row.mid_l_prev > row.mid_l:
        return row.DIRECTION_prev
    return 0

def get_entry_stop(row):
    if row.SIGNAL == 1:
        return (row.RANGE_prev * ENTRY_PRC) + row.ask_h_prev
    elif row.SIGNAL == -1:
        return row.bid_l_prev - (row.RANGE_prev * ENTRY_PRC)
    else:
        return 0
    
def get_stop_loss(row):
    if row.SIGNAL == 1:
        return row.ENTRY - (row.RANGE_prev * SLOSS)
    if row.SIGNAL == -1:
        return row.ENTRY + (row.RANGE_prev * SLOSS)
    else:
        return 0
    
def get_take_profit(row):
    if row.SIGNAL == 1:
        return row.ENTRY + (row.RANGE_prev * TPROFIT)
    if row.SIGNAL == -1:
        return row.ENTRY - (row.RANGE_prev * TPROFIT)
    else:
        return 0

## Additional column capture/derivation.

* The range of a candle is the difference between the high and low price.
* To track whether the high and low and enclosed in previous candle, we need high and low values from previous.



In [54]:
#TODO:  Create a new dataframe/sheet to capture Data Dictionary and some metadata...

df = df_raw.copy()
df['RANGE'] = df.mid_h - df.mid_l
df['mid_h_prev'] = df.mid_h.shift(1)
df['mid_l_prev'] = df.mid_l.shift(1)
df['ask_h_prev'] = df.ask_h.shift(1)
df['bid_l_prev'] = df.bid_l.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)
df.head()


,Unnamed: 0,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,mid_l,...,ask_c,RANGE,mid_h_prev,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL
0,1,2020-01-01 22:05:00+00:00,28,144.039,144.048,144.020,144.030,144.114,144.118,144.084,...,144.139,0.034,144.212,144.102,144.287,144.031,0.110,-1,-1,-1
1,2,2020-01-01 22:10:00+00:00,55,144.030,144.032,143.769,144.025,144.080,144.105,143.844,...,144.115,0.261,144.118,144.084,144.188,144.020,0.034,-1,-1,-1
2,3,2020-01-01 22:15:00+00:00,96,144.029,144.034,143.861,143.917,144.074,144.079,143.936,...,144.067,0.143,144.105,143.844,144.180,143.769,0.261,-1,-1,0
3,4,2020-01-01 22:20:00+00:00,52,143.922,143.956,143.904,143.956,143.997,144.031,143.979,...,144.106,0.052,144.079,143.936,144.145,143.861,0.143,1,-1,0
4,5,2020-01-01 22:25:00+00:00,311,143.950,144.007,143.923,143.998,144.025,144.082,143.998,...,144.148,0.084,144.031,143.979,144.106,143.904,0.052,1,1,0


Capture how many rows in our dataframe, and count how many buy/sell signals we're getting.

In [55]:
print(f"total number of rows:  {df.shape[0]}")
df.groupby(by="SIGNAL").count()

total number of rows:  224096


,Unnamed: 0,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,mid_l,...,ask_l,ask_c,RANGE,mid_h_prev,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev
SIGNAL,,,,,,,,,,,,,,,,,,,,,
-1,59778,59778,59778,59778,59778,59778,59778,59778,59778,59778,...,59778,59778,59778,59778,59778,59778,59778,59778,59778,59778
0,143952,143952,143952,143952,143952,143952,143952,143952,143952,143952,...,143952,143952,143952,143952,143952,143952,143952,143952,143952,143952
1,20366,20366,20366,20366,20366,20366,20366,20366,20366,20366,...,20366,20366,20366,20366,20366,20366,20366,20366,20366,20366


In [56]:
df['ENTRY'] = df.apply(get_entry_stop, axis=1)
df['STOPLOSS'] = df.apply(get_stop_loss, axis=1)
df['TAKEPROFIT'] = df.apply(get_take_profit, axis=1)

df_plot = df.iloc[0:60]
df_buys = df_plot[df_plot.SIGNAL == 1]
df_sells = df_plot[df_plot.SIGNAL == -1]

plot_cols = ['ENTRY', 'STOPLOSS', 'TAKEPROFIT']





In [57]:
df['ENTRY'] = df.apply(get_entry_stop, axis=1)
df['STOPLOSS'] = df.apply(get_stop_loss, axis=1)
df['TAKEPROFIT'] = df.apply(get_take_profit, axis=1)
df[df.SIGNAL==1].head()


,Unnamed: 0,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,mid_l,...,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
16,17,2020-01-01 23:25:00+00:00,71,144.042,144.053,144.022,144.046,144.068,144.076,144.046,...,144.058,144.117,144.033,0.032,-1,1,1,144.1202,144.1074,144.1458
28,29,2020-01-02 00:25:00+00:00,70,144.023,144.035,144.004,144.011,144.042,144.052,144.022,...,144.026,144.141,144.010,0.094,-1,1,1,144.1504,144.1128,144.2256
38,39,2020-01-02 01:15:00+00:00,127,144.129,144.132,144.096,144.100,144.152,144.152,144.114,...,144.118,144.203,144.103,0.069,-1,1,1,144.2099,144.1823,144.2651
41,42,2020-01-02 01:30:00+00:00,102,144.118,144.121,144.074,144.091,144.132,144.136,144.090,...,144.122,144.161,144.105,0.022,-1,1,1,144.1632,144.1544,144.1808
53,54,2020-01-02 02:30:00+00:00,57,143.940,143.941,143.915,143.926,143.954,143.956,143.932,...,143.942,143.994,143.929,0.038,-1,1,1,143.9978,143.9826,144.0282


In [58]:
df_buys.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 16 to 53
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      5 non-null      int64  
 1   time            5 non-null      object 
 2   volume          5 non-null      int64  
 3   bid_o           5 non-null      float64
 4   bid_h           5 non-null      float64
 5   bid_l           5 non-null      float64
 6   bid_c           5 non-null      float64
 7   mid_o           5 non-null      float64
 8   mid_h           5 non-null      float64
 9   mid_l           5 non-null      float64
 10  mid_c           5 non-null      float64
 11  ask_o           5 non-null      float64
 12  ask_h           5 non-null      float64
 13  ask_l           5 non-null      float64
 14  ask_c           5 non-null      float64
 15  RANGE           5 non-null      float64
 16  mid_h_prev      5 non-null      float64
 17  mid_l_prev      5 non-null      float

In [59]:
class Trade():
    def __init__(self, row):
        self.candle_date = row.time
        self.direction = row.SIGNAL
        self.entry = row.ENTRY
        self.TP = row.TAKEPROFIT
        self.SL = row.STOPLOSS
        self.running = False
        self.result = None
        self.stopped = None

    def update(self, row):
        if self.running == True:
            self.update_result(row)
        else:
            self.check_entry(row)

    def check_entry(self, row):
        if self.direction == 1 and row.mid_c >= self.entry or self.direction == -1 and row.mid_c <= self.entry:
            self.index = row.name
            self.opened = row.time
            self.running = True

    def update_result(self, row):
        if self.direction == 1:
            if row.mid_c >= self.TP:
                self.result = 2.0
            elif row.mid_c <= self.SL:
                self.result = -1.0
        else:
            if row.mid_c <= self.TP:
                self.result = 2.0
            elif row.mid_c >= self.SL:
                self.result = -1.0

        if self.result is not None:
            self.running = False
            self.stopped = row.time



In [60]:
open_trades = []
closed_trades = []

for index, row in df.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.stopped is not None:
            closed_trades.append(ot)

    open_trades = [x for x in open_trades if x.stopped is None]

    if row.SIGNAL != 0:
        open_trades = [x for x in open_trades if x.running == True]
        open_trades.append(Trade(row))


    

In [61]:
len(closed_trades)

32776

In [62]:
df_trades = pd.DataFrame.from_dict([vars(x) for x in closed_trades])
df_trades.head()

,candle_date,direction,entry,TP,SL,running,result,stopped,index,opened
0,2020-01-01 22:10:00+00:00,-1,144.0166,143.9894,144.0302,False,-1.0,2020-01-01 22:20:00+00:00,2,2020-01-01 22:15:00+00:00
1,2020-01-02 00:25:00+00:00,1,144.1504,144.2256,144.1128,False,-1.0,2020-01-02 01:30:00+00:00,35,2020-01-02 01:00:00+00:00
2,2020-01-02 01:40:00+00:00,-1,144.0595,144.0395,144.0695,False,2.0,2020-01-02 01:50:00+00:00,44,2020-01-02 01:45:00+00:00
3,2020-01-02 01:45:00+00:00,-1,144.0454,144.0166,144.0598,False,2.0,2020-01-02 02:10:00+00:00,45,2020-01-02 01:50:00+00:00
4,2020-01-02 02:05:00+00:00,-1,144.0090,143.9770,144.0250,False,2.0,2020-01-02 02:15:00+00:00,49,2020-01-02 02:10:00+00:00


In [63]:
df_trades.result.sum()

27806.0

In [64]:
df[df.SIGNAL==1].head()

,Unnamed: 0,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,mid_l,...,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
16,17,2020-01-01 23:25:00+00:00,71,144.042,144.053,144.022,144.046,144.068,144.076,144.046,...,144.058,144.117,144.033,0.032,-1,1,1,144.1202,144.1074,144.1458
28,29,2020-01-02 00:25:00+00:00,70,144.023,144.035,144.004,144.011,144.042,144.052,144.022,...,144.026,144.141,144.010,0.094,-1,1,1,144.1504,144.1128,144.2256
38,39,2020-01-02 01:15:00+00:00,127,144.129,144.132,144.096,144.100,144.152,144.152,144.114,...,144.118,144.203,144.103,0.069,-1,1,1,144.2099,144.1823,144.2651
41,42,2020-01-02 01:30:00+00:00,102,144.118,144.121,144.074,144.091,144.132,144.136,144.090,...,144.122,144.161,144.105,0.022,-1,1,1,144.1632,144.1544,144.1808
53,54,2020-01-02 02:30:00+00:00,57,143.940,143.941,143.915,143.926,143.954,143.956,143.932,...,143.942,143.994,143.929,0.038,-1,1,1,143.9978,143.9826,144.0282


In [65]:
df[df.SIGNAL==1].to_csv("./Data/Pairs/USD_JPY_H4_trades.csv")

In [68]:
plot_candles_mode(df_plot, "markers")

In [67]:
plot_candles_mode(df_plot, "lines")